# Notebook 02: Data Exploration - Help Center Intelligence Analyzer

**Author:** Hector Carbajal  
**Version:** 1.1  
**Last Updated:** 2026-02

---

## Purpose

This notebook performs **exploratory data analysis** on the Help Center macro library:
1. Analyze ticket volume and macro usage patterns
2. Explore effectiveness distributions by category
3. Identify high-performing "hidden gems" and low-performers
4. Validate clustering quality for macro consolidation

## Inputs
- `data/processed/macro_scores.csv` - Macro effectiveness scores
- `data/processed/macro_clusters.csv` - NLP cluster assignments
- `data/processed/cluster_summary.csv` - Cluster-level metrics

## Key Business Questions
- Which macros should be promoted to higher visibility?
- Which macros are candidates for deprecation or rewriting?
- Are there redundant macro clusters that should be consolidated?

## Table of Contents
1. [Setup & Data Loading](#setup)
2. [Data Quality Audit](#audit)
3. [Usage Distribution Analysis](#usage)
4. [Effectiveness Analysis](#effectiveness)
5. [Cluster Analysis](#clusters)
6. [💡 Key Findings Summary](#findings)

---

In [1]:
# Setup
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.config import (
    TICKETS_FEATURES_FILE,
    MACRO_CLUSTERS_FILE,
    CLUSTER_SUMMARY_FILE
)

# Set Plotly Template
px.defaults.template = "plotly_white"

<a id="audit"></a>
## 1. Data Quality Audit
*Verification of data integrity before analysis.*

In [2]:
# Load data
tickets = pd.read_csv(TICKETS_FEATURES_FILE)
macros = pd.read_csv(MACRO_CLUSTERS_FILE)
clusters = pd.read_csv(CLUSTER_SUMMARY_FILE)

print("✅ DATA AUDIT SUMMARY:")
print(f"- Records: {len(tickets):,} tickets, {len(macros):,} macros, {len(clusters):,} clusters")
print(f"- Null Scores: {tickets['csat_score'].isna().sum()} (None)")
print(f"- Uniqueness: {macros['macro_id'].is_unique} (Verified)")
print(f"- Logical Consistency: {len(tickets[tickets['total_handle_time_minutes'] < 0])} negative times (None)")

✅ DATA AUDIT SUMMARY:
- Records: 10,000 tickets, 150 macros, 12 clusters
- Null Scores: 0 (None)
- Uniqueness: True (Verified)
- Logical Consistency: 0 negative times (None)


<a id="usage"></a>
## 2. Usage & Outcome Analysis
*How does macro adoption correlate with ticket resolution quality?*

In [3]:
# Outcome Comparison
comparison = tickets.groupby('is_macro_used').agg({
    'csat_score': 'mean',
    'total_handle_time_minutes': 'mean',
    'reopens_count': lambda x: (x > 0).mean()
}).round(2)
comparison.columns = ['Avg CSAT', 'Avg Handle Time (min)', 'Reopen Rate']
comparison.index = ['Manual Response', 'Macro Used']

print("📊 MACRO IMPACT ON KEY METRICS:")
display(comparison)

📊 MACRO IMPACT ON KEY METRICS:


,Avg CSAT,Avg Handle Time (min),Reopen Rate
Manual Response,2.93,31.15,0.15
Macro Used,3.01,30.28,0.16


**Insight:** Macros successfully drive higher **CSAT** (+0.08) and lower **Handle Time** (-0.9 min). However, they show a slightly higher **Reopen Rate** (0.16 vs 0.15), suggesting that while macros speed up resolution, they may occasionally miss nuances required for a 'one-touch' fix.

In [4]:
# Contact Driver Distribution
fig = px.bar(
    tickets['contact_driver'].value_counts().reset_index(),
    x='contact_driver',
    y='count',
    title='Ticket Volume by Contact Driver',
    labels={'contact_driver': 'Category', 'count': 'Volume'},
    color='count',
    color_continuous_scale='Viridis'
)
fig.update_layout(showlegend=False)
fig.show()

<a id="effectiveness"></a>
## 3. Macro Effectiveness Deep-Dive
*Segmenting the library to identify high-value assets and legacy bloat.*

In [5]:
# Category Performance Grid
category_stats = macros[macros['has_sufficient_usage']].groupby('category').agg({
    'macro_effectiveness_index': 'mean',
    'usage_count': 'sum',
    'macro_id': 'count'
}).round(1)
category_stats.columns = ['Avg Effectiveness', 'Total Usage', 'Macro Count']

print("📊 PERFORMANCE BY BUSINESS CATEGORY:")
display(category_stats.sort_values('Avg Effectiveness', ascending=False))

📊 PERFORMANCE BY BUSINESS CATEGORY:


,Avg Effectiveness,Total Usage,Macro Count
category,,,
account,66.4,2110,30
product,64.3,1362,21
policy,61.4,1069,20
billing,53.7,4037,32
technical,51.7,3155,25
escalation,41.9,228,3


In [6]:
# Cluster overview
fig = px.scatter(
    clusters,
    x='num_macros',
    y='avg_effectiveness',
    size='total_usage',
    color='consolidation_candidate',
    hover_name='cluster_label',
    title='Cluster Efficiency Matrix (Size = Total Usage)',
    labels={
        'num_macros': 'Density (Macro Count)',
        'avg_effectiveness': 'Effectiveness Score',
        'consolidation_candidate': 'Redundancy Flag'
    }
)
fig.show()

**Analysis:** Clusters in the top-left are high-performing lean categories. Clusters in the bottom-right are likely candidates for consolidation or archiving due to low effectiveness and high macro density.

<a id="findings"></a>
## 💡 Key Findings & Recommendations
**Executive Summary for Stakeholders**

In [8]:
# Extract dynamic values for insights
macro_impact = comparison.loc['Macro Used'] - comparison.loc['Manual Response']
gems = macros[macros['macro_category'] == 'Underused Gem']
low_perf = macros[macros['macro_category'] == 'Low Effectiveness']
redundant = clusters[clusters['consolidation_candidate']]

print("="*80)
print("🚀 REVENUE & EFFICIENCY RECOMMENDATIONS")
print("="*80)

print(f"\n1. PROMOTION OPPORTUNITY:")
print(f"   Identified {len(gems)} 'Underused Gems' with 80%+ effectiveness index but low adoption.")
print(f"   ACTION: Move these to top-level folders in Zendesk to drive agent adoption.")

print(f"\n2. COST SAVINGS:")
ht_delta = macro_impact['Avg Handle Time (min)']
ht_direction = "reducing" if ht_delta < 0 else "increasing"
print(f"   Macros are {ht_direction} Handle Time by {abs(ht_delta):.1f} minutes per ticket.")
print(f"   ACTION: Target the top 5 'Manual Response' drivers for new macro creation to save additional capacity.")

print(f"\n3. LIBRARY HYGIENE:")
print(f"   {len(low_perf)} macros fall below effectiveness thresholds.")
print(f"   {len(redundant)} topic clusters show significant semantic overlap.")
print(f"   ACTION: Initiate deprecation cycle for flagged clusters to reduce agent 'Search Fatigue'.")

print("\n" + "="*80)

🚀 REVENUE & EFFICIENCY RECOMMENDATIONS

1. PROMOTION OPPORTUNITY:
   Identified 25 'Underused Gems' with 80%+ effectiveness index but low adoption.
   ACTION: Move these to top-level folders in Zendesk to drive agent adoption.

2. COST SAVINGS:
   Macros are reducing Handle Time by 0.9 minutes per ticket.
   ACTION: Target the top 5 'Manual Response' drivers for new macro creation to save additional capacity.

3. LIBRARY HYGIENE:
   18 macros fall below effectiveness thresholds.
   3 topic clusters show significant semantic overlap.
   ACTION: Initiate deprecation cycle for flagged clusters to reduce agent 'Search Fatigue'.

